# Installation of Required Algorithms

In [1]:
pip install neo

     |████████████████████████████████| 2.2 MB 5.4 MB/s 
     |████████████████████████████████| 83 kB 2.7 MB/s 
  Created wheel for neo: filename=neo-0.10.0-py3-none-any.whl size=549615 sha256=145869001acf533b809ddfe5fb41697d41b0b01072c607f3fdaa27cebe4f547f
  Stored in directory: /root/.cache/pip/wheels/f9/44/07/60b235ef9fdf0ab018a8bbff3305266aae49cbdbf8187a49a8
  Created wheel for quantities: filename=quantities-0.12.4-py3-none-any.whl size=79181 sha256=a0ea8708575f353aac7e3993baf33e1e481a697e3626a3eab0206cd77205f1a0
  Stored in directory: /root/.cache/pip/wheels/3c/aa/86/9ea0f88649199b4ba7c457b7acdc6924c78466ea056b466f13
Successfully built neo quantities


In [2]:
pip install quantities

In [3]:
pip install elephant

     |████████████████████████████████| 2.0 MB 4.4 MB/s 
     |████████████████████████████████| 28.5 MB 30 kB/s 
  Created wheel for elephant: filename=elephant-0.10.0-py3-none-any.whl size=802402 sha256=bb6756db8681e3f596d139a4cff6d33b63cad2a40f88c5458cf88a5ee376002e
  Stored in directory: /root/.cache/pip/wheels/44/f3/14/d21fb65ed2f0a28ef1733169bbee92daa9d78220a0320c624f
Successfully built elephant
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [4]:
pip install elephant[extras]

     |████████████████████████████████| 9.5 MB 108 kB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [5]:
pip install --upgrade elephant

# Importing the libraries

In [6]:

import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.sparse import random
from scipy import stats
from quantities import ms, s, Hz
from elephant.spike_train_generation import homogeneous_poisson_process, homogeneous_gamma_process
from elephant.statistics import mean_firing_rate, time_histogram, instantaneous_rate
from random import choices

np.random.seed(42)



# Defining the variables

In [7]:

num_neuron = 3000
sampling_period = 50*ms
dT = 0.05
T = 40

t = np.linspace(0, T+dT, int(T/dT))

# Creating Neural Network Structure

In [8]:
# Obtaining firing rate covariance matrix

def cov_matrix(num_neuron = 1000, rate = 10*Hz, t_start = 0*ms, t_stop = 10000*ms, sampling_period = 50*ms):

  spiketrain = []
  inst_rate = []

  for neuron in np.arange(num_neuron):
    spiketrain.append(homogeneous_poisson_process(rate=rate, t_start=t_start, t_stop=t_stop))
    inst_rate.append(instantaneous_rate(spiketrain[neuron], sampling_period=sampling_period))

  neuron = 0

  fr_stack = inst_rate[neuron].T[0]

  while neuron<len(spiketrain)-1:
    fr_stack = np.vstack((fr_stack, inst_rate[neuron+1].T[0].T))
    neuron += 1

  cov = np.cov(fr_stack)

  return cov, fr_stack

In [9]:
# External input equation

def external_input(num_neuron = 1000, b = 0.0002, T = 40, dT = 0.1, sigma = 1):

  t = np.linspace(0, T+dT, int(T/dT))

  eta = np.random.normal(loc=0, scale = sigma, size = (num_neuron, int(T/dT)))

  ext_input = b + eta

  return ext_input




In [10]:
# Defining random variable parameters

normal_rvs = stats.norm(loc=1, scale=1).rvs

uniform_rvs = stats.uniform.rvs

In [11]:
# Network Construction

def within_network(num_neuron = 1000, cluster_num = 3, cluster_sizes = [250,250,500] , within_conn_prob = 0.6):

  if num_neuron != np.sum(cluster_sizes):
    raise Exception("Number of neurons must be equal with sum of cluster sizes")

  if len(cluster_sizes) != cluster_num:
    raise Exception("len(cluster_sizes) must be equal with cluster_num")

  if within_conn_prob > 1 or within_conn_prob<0:
    raise Exception("Probabilities must be from 0 to 1")


  clusters={}

  for num in np.arange(cluster_num):
    non_symm = random(cluster_sizes[num], cluster_sizes[num], density=within_conn_prob, data_rvs=normal_rvs).toarray()
    clusters["%s" %num] = (non_symm + non_symm.T)/2

  return clusters



def sparse_network(num_neuron = 1000, cluster_num = 3, cluster_sizes = [250,250,500] , sparse_conn = 5000):

  a_network = np.zeros((num_neuron,num_neuron))
  index_neuron=0

  for num in np.arange(cluster_num):
    a_network[index_neuron:index_neuron + cluster_sizes[num], index_neuron:index_neuron + cluster_sizes[num]] = np.ones((cluster_sizes[num],cluster_sizes[num]))
    index_neuron += cluster_sizes[num]

  empty_where = np.where(a_network==0)
  empty_conn_num = empty_where[0].size

  rng = np.random.default_rng()
  sparse_random = rng.choice(np.arange(empty_conn_num), sparse_conn, replace = False)
  sparse_where = [empty_where[0][sparse_random],empty_where[1][sparse_random]]

  sparse_conn_power = np.random.rand(sparse_conn)

  a_network[sparse_where] = sparse_conn_power
  sparse_network = (a_network + a_network.T)/2

  return sparse_network



def connectivity_matrix(clusters, sparse_network):

  if type(clusters) != dict:
    raise Exception("The data type of clusters must be a dictionary (dict)")

  conn_matrix = sparse_network
  index_neuron = 0

  for num in np.arange(len(clusters)):
    conn_matrix[index_neuron:index_neuron + clusters["%s" %num].shape[0], index_neuron:index_neuron + clusters["%s" %num].shape[0]] = clusters["%s" %num]
    index_neuron += clusters["%s" %num].shape[0]

  return conn_matrix

  

In [12]:
def get_leaking(num_neuron = 1000):

  D = np.random.rand(num_neuron, num_neuron)
  D = np.diag(np.diag(D))

  return D

In [13]:
def get_probs(cov, conn_matrix, K = 0.5):

  height, weight = cov.shape

  probs = np.zeros((height, weight))

  for h in np.arange(height):
    for w in np.arange(weight):

      if conn_matrix[h,w] > 0:
        probs[h,w] = K * conn_matrix[h,w] * (cov[h,h] + cov[w,w] - 2*cov[h,w])
      elif conn_matrix[h,w] < 0:
        probs[h,w] = K * np.absolute(conn_matrix[h,w]) * (cov[h,h] + cov[w,w] + 2*cov[h,w])

  while np.where(probs>1)[0].shape != np.array([0]):

    probs = probs/10
    K = K/10
    print("new K value is: ", K)


  return probs

In [14]:
def a_sparse(A, probs):

  height, weight = A.shape

  A_sparse = np.zeros((height,weight))

  for h in np.arange(height):
    for w in np.arange(weight):
      A_sparse[h,w] = np.random.choice([A[h,w]/probs[h,w], 0], 1, [probs[h,w], 1-probs[h,w]])
      if A_sparse[h,w] == np.inf:
        A_sparse[h,w] = 0
      elif A_sparse[h,w] == -np.inf:
        A_sparse[h,w] = 0

  A_sparse = np.nan_to_num(A_sparse)

  return A_sparse

In [19]:
# x is a vector (N x n_sampling) of firing rates which changes with time
# D is a N x N diagonal matrix, resembling leak current from every neuron
# W is the N x N connection matrix, W_ij shows connection strength from jth neuron to ith neuron
# b(t) is the external input vector (N x 1)


b=external_input(num_neuron=3000, b=0.0002, T=40, dT=0.05, sigma = 1)

sparse = sparse_network(num_neuron = 3000, 
                        cluster_num = 4, 
                        cluster_sizes = [100,100,100,2700], 
                        sparse_conn = 5000)

network=within_network(num_neuron=3000, 
                       cluster_num=4, 
                       cluster_sizes=[100,100,100,2700])

W = connectivity_matrix(network, sparse)

D = get_leaking(3000)

cov, fr_stack = cov_matrix(num_neuron = 3000,
                 rate = 10*Hz,
                 t_start= 0*ms, 
                 t_stop = 10000*ms, 
                 sampling_period = 50*ms)

probs = get_probs(cov = cov, conn_matrix = W, K=0.5)

A = -D + W

A_sparse = a_sparse(A,probs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


new K value is:  0.05
new K value is:  0.005


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


In [15]:
def preserve_diag(A, A_sparse):

  H,W = A_sparse.shape

  for h in np.arange(H):

    A_sparse[h,h] = A[h,h]

  return A_sparse

In [ ]:
A_sparse = preserve_diag(A, A_sparse)

In [16]:
# Since quantities library does not support matrix multiplication, I have written matrix multiplication algorithm

def matrix_multip(A,x):

  H,W = A.shape
  Ax = np.zeros((H))

  for h in np.arange(H):
    Ax[h] = np.sum(A[h,:] * x)

  return Ax

In [17]:
def dxdt(D, W, A_sparse, b, x, dt):

  num_neuron, num_sample = b.shape
  A = -D + W
  A_sparse = A_sparse.reshape((num_neuron, num_neuron,1))

  for sample in np.arange(num_sample):
    
    dxdt = matrix_multip(A_sparse[:,:,sample], x[:,-1]) + b[:,sample]
    dx = dxdt * dt
    current_x = x[:,-1] + dx
    current_x = current_x.reshape((num_neuron,1))
    x = np.hstack((x, current_x))

    cov = np.cov(x)

    probs = get_probs(cov, W, K = 0.1)

    current_A_sparse = a_sparse(A_sparse[:,:,sample], probs)
    current_A_sparse = preserve_diag(A, current_A_sparse)

    A_sparse = np.dstack((A_sparse, current_A_sparse))

  return A_sparse

In [ ]:
sparse_stack = dxdt(D, W, A_sparse, b, fr_stack, dT)

new K value is:  0.01
new K value is:  0.001
new K value is:  0.0001
new K value is:  1e-05
new K value is:  1.0000000000000002e-06
new K value is:  1.0000000000000002e-07
new K value is:  1.0000000000000002e-08
new K value is:  1.0000000000000003e-09
new K value is:  1.0000000000000003e-10
new K value is:  1.0000000000000003e-11


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


new K value is:  0.01
new K value is:  0.001
new K value is:  0.0001
new K value is:  1e-05
new K value is:  1.0000000000000002e-06
new K value is:  1.0000000000000002e-07
new K value is:  1.0000000000000002e-08
new K value is:  1.0000000000000003e-09
new K value is:  1.0000000000000003e-10
new K value is:  1.0000000000000003e-11
new K value is:  1.0000000000000002e-12
new K value is:  1.0000000000000002e-13
new K value is:  1.0000000000000002e-14
new K value is:  1e-15
new K value is:  1.0000000000000001e-16
new K value is:  1e-17
new K value is:  1e-18
new K value is:  1.0000000000000001e-19
new K value is:  1.0000000000000001e-20
new K value is:  1.0000000000000001e-21
new K value is:  1e-22
new K value is:  1.0000000000000001e-23
new K value is:  1.0000000000000001e-24
new K value is:  1.0000000000000002e-25
new K value is:  1.0000000000000002e-26
new K value is:  1.0000000000000002e-27
new K value is:  1.0000000000000002e-28
new K value is:  1.0000000000000002e-29
new K value is: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


new K value is:  0.01
new K value is:  0.001
new K value is:  0.0001
new K value is:  1e-05
new K value is:  1.0000000000000002e-06
new K value is:  1.0000000000000002e-07
new K value is:  1.0000000000000002e-08
new K value is:  1.0000000000000003e-09
new K value is:  1.0000000000000003e-10
new K value is:  1.0000000000000003e-11
new K value is:  1.0000000000000002e-12
new K value is:  1.0000000000000002e-13
new K value is:  1.0000000000000002e-14
new K value is:  1e-15
new K value is:  1.0000000000000001e-16
new K value is:  1e-17
new K value is:  1e-18
new K value is:  1.0000000000000001e-19
new K value is:  1.0000000000000001e-20
new K value is:  1.0000000000000001e-21
new K value is:  1e-22
new K value is:  1.0000000000000001e-23
new K value is:  1.0000000000000001e-24
new K value is:  1.0000000000000002e-25
new K value is:  1.0000000000000002e-26
new K value is:  1.0000000000000002e-27
new K value is:  1.0000000000000002e-28
new K value is:  1.0000000000000002e-29
new K value is: 

KeyboardInterrupt: ignored

In [21]:
# x is a vector (N x n_sampling) of firing rates which changes with time
# D is a N x N diagonal matrix, resembling leak current from every neuron
# W is the N x N connection matrix, W_ij shows connection strength from jth neuron to ith neuron
# b(t) is the external input vector (N x 1)


b=external_input(num_neuron=3000, b=0.0002, T=40, dT=0.05, sigma = 1)

sparse = sparse_network(num_neuron = 3000, 
                        cluster_num = 4, 
                        cluster_sizes = [100,100,100,2700], 
                        sparse_conn = 5000)

network=within_network(num_neuron=3000, 
                       cluster_num=4, 
                       cluster_sizes=[100,100,100,2700])

W = connectivity_matrix(network, sparse)

D = get_leaking(3000)

cov, fr_stack = cov_matrix(num_neuron = 3000,
                 rate = 5*Hz,
                 t_start= 0*ms, 
                 t_stop = 10000*ms, 
                 sampling_period = 50*ms)

probs = get_probs(cov = cov, conn_matrix = W, K=0.5)

A = -D + W

A_sparse = a_sparse(A,probs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


new K value is:  0.05
new K value is:  0.005


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


In [22]:
A_sparse = preserve_diag(A, A_sparse)

In [23]:
sparse_stack = dxdt(D, W, A_sparse, b, fr_stack, dT)

new K value is:  0.01
new K value is:  0.001
new K value is:  0.0001
new K value is:  1e-05
new K value is:  1.0000000000000002e-06
new K value is:  1.0000000000000002e-07
new K value is:  1.0000000000000002e-08
new K value is:  1.0000000000000003e-09
new K value is:  1.0000000000000003e-10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


new K value is:  0.01
new K value is:  0.001
new K value is:  0.0001
new K value is:  1e-05
new K value is:  1.0000000000000002e-06
new K value is:  1.0000000000000002e-07
new K value is:  1.0000000000000002e-08
new K value is:  1.0000000000000003e-09
new K value is:  1.0000000000000003e-10
new K value is:  1.0000000000000003e-11
new K value is:  1.0000000000000002e-12
new K value is:  1.0000000000000002e-13
new K value is:  1.0000000000000002e-14
new K value is:  1e-15
new K value is:  1.0000000000000001e-16
new K value is:  1e-17
new K value is:  1e-18
new K value is:  1.0000000000000001e-19
new K value is:  1.0000000000000001e-20
new K value is:  1.0000000000000001e-21
new K value is:  1e-22
new K value is:  1.0000000000000001e-23
new K value is:  1.0000000000000001e-24
new K value is:  1.0000000000000002e-25
new K value is:  1.0000000000000002e-26
new K value is:  1.0000000000000002e-27
new K value is:  1.0000000000000002e-28
new K value is:  1.0000000000000002e-29
new K value is: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in double_scalars
  del sys.path[0]


Görüntülenen çıkış son 5000 satıra kısaltıldı.
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K value is:  0.0
new K va

KeyboardInterrupt: ignored